## RTA Monitor

1. Define the number of seconds you would like to monitor the process for 
2. Define any short comments about the process 

In [17]:
# remeber, this number is in seconds!
READ_FOR = 2
# COMMENT = ""

In [18]:
from datetime import datetime
date = datetime.now().strftime("%Y_%m_%d-%I-%M_%p")
print("Today: {}".format(date))

Today: 2021_08_19-09-09_PM


Where should I save plot data?

In [19]:
from pathlib import Path 

data_path = Path("../")

Great. Here are some useful `python` libraries that we will be using. Please help me import them by clicking the run button next to the cell.

In [20]:
import plotly.graph_objs as go
import h5py
import numpy as np
import matplotlib.pyplot as plt 
from SerialReader import SerialReaderPlotter

s = SerialReaderPlotter("COM8", baudrate=9600)

## Live Plot
The next cell creates a plot window where our temperature versus time trace will go. Every time you run the following cell, a new graph will be created.

In [21]:
fig = go.FigureWidget(data=[])
fig.update_layout(
    template="plotly_white",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (C)",
)

FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'T…

## Begin
When you are ready, click run in the following cell to start the monitoring process.

In [22]:
"""
In this cell, I left to you the convenience of allowing to other things 
while the serial reader is reading.
"""
s.start_reading(READ_FOR)
fig.update_layout(xaxis_range=[0,READ_FOR])
s.start_writing(fig)
s.writer.join()

(Reader   ) Starting
(Plotter  ) Starting
(Reader   ) Reading 18.0
(Reader   ) Exiting
(Plotter  ) Exiting


## Saving Data

In [24]:
for i, scatter in enumerate(fig.data):
    filename = data_path/Path("RTA_Run_{}_".format(i) + date + ".h5")

    hf = h5py.File(filename, 'w')
    hf.create_dataset('Time', data=np.array(scatter.x, dtype=float))
    hf.create_dataset('Temperature', data=np.array(scatter.y, dtype=float))
    hf.close()